In [1]:
!pip install --no-index --find-links=/kaggle/input/lmsyspackages1/ --no-build-isolation /kaggle/input/lmsyspackages1/transformers-4.44.0.dev0.zip
!pip install --no-index --find-links=/kaggle/input/lmsyspackages1/ --no-build-isolation /kaggle/input/lmsyspackages1/bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl

Looking in links: /kaggle/input/lmsyspackages1/
Processing /kaggle/input/lmsyspackages1/transformers-4.44.0.dev0.zip
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.44.0.dev0-py3-none-any.whl size=9405504 sha256=2037b362ed68774c6b5282a102bf44080623f8692f1b0c4bc3b0e3db3a0ddb1b
  Stored in directory: /root/.cache/pip/wheels/70/b9/f8/ca2eaba14f0638d458336da55845ab4c5db8c95d2436fdbc82
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
Looking in links: /kaggle/input/lmsyspackages1/
Processing /kaggle/input/lmsyspackages1/bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl


In [2]:
# Import necessary libraries for loading datasets, progress tracking, and data manipulation
from datasets import load_dataset
from tqdm.notebook import tqdm
import torch
import pandas as pd
from glob import glob
import numpy as np
import os

# Import transformers for model and tokenizer, itertools for efficient looping, and random for randomness
from transformers import AutoModelForCausalLM, AutoTokenizer
import itertools
import random
import argparse
import os
import pandas as pd
import numpy as np
from string import Template
from pathlib import Path
import time
import torch
from tqdm.auto import tqdm
import gc

# The code imports multiple modules for different functionalities:
# - `datasets` for loading and processing datasets.
# - `tqdm` for progress bars.
# - `torch` for PyTorch operations.
# - `pandas` for data manipulation.
# - `glob` for file pattern matching.
# - `numpy` for numerical operations.
# - `os` for interacting with the operating system.
# - `transformers` for model and tokenizer handling from Hugging Face.
# - `itertools` for advanced iteration utilities.
# - `random` for generating random numbers.
# - `argparse` for parsing command-line arguments.
# - `string.Template` for template string substitution.
# - `pathlib.Path` for handling filesystem paths.
# - `time` for time-related functions.
# - `gc` for garbage collection.


In [3]:
# Import necessary libraries
from pathlib import Path  # For handling file paths
import os  # For accessing environment variables
import pandas as pd  # For data manipulation and analysis

# Define the path to the data directory
data_path = Path('/kaggle/input/lmsys-chatbot-arena')

# Check if the environment variable 'KAGGLE_IS_COMPETITION_RERUN' is set
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # If the code is running in a competition rerun, load the test data
    test = pd.read_csv(data_path / 'test.csv')
else:
    # If the code is not running in a competition rerun, load the folds data
    folds = pd.read_csv("/kaggle/input/lmsysfolds/folds.csv")
    # Load the training data
    test = pd.read_csv(data_path / 'train.csv')
    # Merge the training data with the folds data on the 'id' column
    test = test.merge(folds, on="id")
    # Filter the data to only include rows where the fold is 0
    test = test.loc[test["fold"] == 0]
    # Select the first 100 rows of the filtered data
    test = test.head(100)

# Replace any missing values with empty strings
test = test.fillna("")

# Display the first few rows of the final dataset
test.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,fold
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0
7,370945,gemini-pro,claude-2.0,"[""\""Bacteria is life on Mars but a heartbeat i...","[""Dune""]","[""This quote seems to be referencing the debat...",0,1,0,0
13,738614,mpt-30b-chat,gpt4all-13b-snoozy,"[""What's the best first move in tic-tac-toe?"",...","[""The best first move in tic-tac-toe is often ...","[""The best first move in tic-tac-toe is to pla...",1,0,0,0
14,862324,vicuna-13b,koala-13b,"[""Write me a poem in urdu in the style of Iqba...","[""Jism ko kisi ka paisa nahi,\nWo kisi ka jaan...","[""In the realm of selflessness dwells the true...",0,0,1,0
29,1827787,deepseek-llm-67b-chat,gpt-4-1106-preview,"[""Given a passage and some supplementary infor...","[""Refined passage: \nThe person in the photo i...","[""Refined passage: \nThe person in the photo i...",0,0,1,0


In [4]:
import json
test["prompt"] = test["prompt"].apply(lambda x: json.loads(x))
test["response_a"] = test["response_a"].apply(lambda x: json.loads(x))
test["response_b"] = test["response_b"].apply(lambda x: json.loads(x))

In [5]:
# Initialize an empty list to store the full prompts
full_prompts = []

# Iterate over each row in the test DataFrame
for idx, row in test.iterrows():
    # Initialize an empty string to build the full prompt for each row
    full_prompt = ""

    # Iterate over each prompt in the row
    for i in range(len(row["prompt"])):
        # Add turn information to the prompt
        full_prompt += f"## Turn {i + 1}\n\n"
        
        # Get response A, handling None values
        if row["response_a"][i] is None:
            res_a = "None"
        else:
            res_a = row["response_a"][i]
        
        # Get response B, handling None values
        if row["response_b"][i] is None:
            res_b = "None"
        else:
            res_b = row["response_b"][i]
        
        # Construct the prompt with responses A and B
        full_prompt += "### Prompt\n" + row["prompt"][i] + "\n\n### Response A\n" + res_a + "\n\n### Response B\n" + res_b + "\n\n"
    
    # Add the final question to the prompt
    full_prompt += "## Which response is better?"
    
    # Append the constructed prompt to the list
    full_prompts.append(full_prompt)

# Initialize another list to store the full prompts for test-time augmentation (TTA)
full_prompts_tta = []

# Iterate over each row in the test DataFrame again for TTA
for idx, row in test.iterrows():
    # Initialize an empty string to build the full prompt for each row
    full_prompt = ""

    # Iterate over each prompt in the row
    for i in range(len(row["prompt"])):
        # Add turn information to the prompt
        full_prompt += f"## Turn {i + 1}\n\n"
        
        # Get response A, handling None values, but switch response A and B compared to the previous iteration
        if row["response_a"][i] is None:
            res_a = "None"
        else:
            res_a = row["response_a"][i]
        
        # Get response B, handling None values
        if row["response_b"][i] is None:
            res_b = "None"
        else:
            res_b = row["response_b"][i]
        
        # Construct the prompt with responses A and B switched
        full_prompt += "### Prompt\n" + row["prompt"][i] + "\n\n### Response A\n" + res_b + "\n\n### Response B\n" + res_a + "\n\n"
    
    # Add the final question to the prompt
    full_prompt += "## Which response is better?"
    
    # Append the constructed prompt to the TTA list
    full_prompts_tta.append(full_prompt)

# Assign the full prompts to a new column in the test DataFrame
test["full_prompt"] = full_prompts

# Save the modified DataFrame to a parquet file
test.to_parquet("test.pq")


In [6]:
%%writefile process.py
import argparse
import os
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Gemma2Model
import torch.nn.functional as F
from torch.cuda.amp import autocast

# Setting environment variables for transformers and PyTorch
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def sort_and_batch_samples(data, tokenizer, max_length):
    # Tokenizing the data without adding special tokens and with truncation
    encoded = tokenizer(data['full_prompt'].tolist(), add_special_tokens=False, truncation=True, max_length=max_length, padding=False, return_tensors=None)
    
    # Calculate lengths of tokenized inputs
    lengths = [len(ids) for ids in encoded['input_ids']]
    
    # Sort indices by length in descending order
    sorted_indices = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
    sorted_data = data.iloc[sorted_indices].reset_index(drop=True)
    sorted_encoded = {k: [v[i] for i in sorted_indices] for k, v in encoded.items()}
    sorted_lengths = [lengths[i] for i in sorted_indices]
    
    # Map original indices to sorted indices
    position_map = {i: idx for idx, i in enumerate(sorted_indices)}
    
    return sorted_data, sorted_encoded, sorted_lengths, position_map

def dynamic_batch(sorted_data, sorted_lengths, max_tokens, tokenizer, device):
    # Function to dynamically batch the data based on token lengths and max tokens per batch
    batches = []
    current_batch = []
    current_length = 0
    
    for i, length in enumerate(sorted_lengths):
        if current_length + length > max_tokens and current_batch:
            # If current batch exceeds max_tokens, yield the batch
            yield tokenizer(sorted_data.loc[current_batch, 'full_prompt'].tolist(), 
                            return_tensors="pt", 
                            padding=True, 
                            truncation=True, 
                            max_length=max(sorted_lengths[j] for j in current_batch)
                           ).to(device), current_batch
            current_batch = []
            current_length = 0
        
        current_batch.append(i)
        current_length += length
    
    if current_batch:
        # Yield the last batch if there are remaining items
        yield tokenizer(sorted_data.loc[current_batch, 'full_prompt'].tolist(), 
                        return_tensors="pt", 
                        padding=True, 
                        truncation=True, 
                        max_length=max(sorted_lengths[j] for j in current_batch)
                       ).to(device), current_batch

@torch.no_grad()
def process_batches(model, head, batches, position_map, device):
    # Process each batch to get logits
    all_logits = []
    all_indices = []
    for batch, indices in tqdm(batches, desc="Processing batches"):
        with autocast():
            out = model(**batch)
            pooled = out[0][:, -1].to(device)  # Get the last hidden state
            logits = F.softmax(head(pooled.float()), dim=1).cpu()  # Apply the head and softmax
        all_logits.append(logits)
        all_indices.extend(indices)
    
    # Combine logits and reorder them to match the original order
    combined_logits = torch.cat(all_logits).numpy()
    reordered_logits = np.zeros_like(combined_logits)
    for i, idx in enumerate(all_indices):
        reordered_logits[position_map[idx]] = combined_logits[i]
    
    return reordered_logits

def split_data_by_length(data, tokenizer, max_length, num_gpus):
    # Tokenize and calculate lengths of the inputs
    encoded = tokenizer(data['full_prompt'].tolist(), add_special_tokens=False, truncation=True, max_length=max_length, padding=False, return_tensors=None)
    lengths = [len(ids) for ids in encoded['input_ids']]
    
    # Sort indices by length in descending order
    sorted_indices = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
    total_length = sum(lengths)
    target_length_per_gpu = total_length // num_gpus
    
    # Split the data across GPUs
    gpu_splits = [[] for _ in range(num_gpus)]
    current_lengths = [0] * num_gpus
    
    for idx in sorted_indices:
        target_gpu = min(range(num_gpus), key=lambda i: current_lengths[i])
        gpu_splits[target_gpu].append(idx)
        current_lengths[target_gpu] += lengths[idx]
    
    return gpu_splits

def main(args):
    # Load the test data
    test = pd.read_parquet("test.pq")
    
    model_name = f"/kaggle/input/{args.model}"
    device_map = {"": args.device} if args.device != "cuda" else "auto"
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.float16,
        bnb_8bit_use_double_quant=False,
    )
    
    # Load the model with the specified configuration
    model = Gemma2Model.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map=device_map,
        trust_remote_code=True,
        quantization_config=bnb_config
    )
    
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=True,
        trust_remote_code=True,
        from_slow=True,
        add_prefix_space=False,
        padding_side="left",
        truncation_side="left"
    )
    
    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.attn_logit_softcapping = None
    
    model_device = args.device
    if args.device == "cuda":
        model_device = "cuda:0"
    
    # Load the classification head weights
    head_weights = torch.load(f"/kaggle/input/{args.model}/classification_head.pth", map_location=model_device)
    head = torch.nn.Linear(model.config.hidden_size, 1, bias=False).to(model_device)
    head.weight.data = head_weights
    
    max_length = 2048  # Adjust as needed
    
    # Split the data across GPUs
    num_gpus = 1 if args.device == "cuda" else 2
    gpu_splits = split_data_by_length(test, tokenizer, max_length, num_gpus)
    
    orig_len = len(test)
    
    if args.device.startswith("cuda:"):
        device_id = int(args.device.split(':')[1])
        current_indices = gpu_splits[device_id]
        test = test.iloc[current_indices].copy()
    elif args.device == "cuda":
        # If using all GPUs, process the entire dataset
        current_indices = gpu_splits[0]
        test = test.iloc[current_indices].copy()

    # Sort and batch the samples
    sorted_data, sorted_encoded, sorted_lengths, position_map = sort_and_batch_samples(test, tokenizer, max_length)
    
    max_tokens = 2048 * 2
    
    # Create dynamic batches
    batches = dynamic_batch(sorted_data, sorted_lengths, max_tokens, tokenizer, args.device)
    
    # Process the batches to get logits
    all_logits = process_batches(model, head, batches, position_map, model_device)
    
    # Reorder logits to match the original data order
    final_logits = np.zeros((orig_len, all_logits.shape[1]))
    for i, orig_idx in enumerate(current_indices):
        final_logits[orig_idx] = all_logits[i]
    
    # Save the logits to file
    if args.device == "cuda":
        np.save("probs", final_logits)
    elif args.device.startswith("cuda:"):
        np.save(f"probs_v{args.device.split(':')[1]}", final_logits)
    else:
        np.save("probs_cpu", final_logits)

if __name__ == "__main__":
    # Set up argument parser and execute main function
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", type=str, required=True)
    parser.add_argument("--device", type=str, required=True)
    args = parser.parse_args()
    main(args)


Writing process.py


In [7]:
%%writefile process.sh
python process.py --model "lmsysmodel0" --device "cuda:0" &
python process.py --model "lmsysmodel0" --device "cuda:1" &
wait 
echo "Done"

Writing process.sh


In [8]:
!sh process.sh

Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████████████| 4/4 [02:12<00:00, 33.18s/it]
Processing batches: 0it [00:00, ?it/s]2024-07-25 14:30:24.689714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 14:30:24.689721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN facto

In [9]:
logits = np.load("probs_v0.npy") + np.load("probs_v1.npy")

In [10]:
test["full_prompt"] = full_prompts_tta
test.to_parquet("test.pq")

In [11]:
!sh process.sh

Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:19<00:00,  4.98s/it]
Processing batches: 0it [00:00, ?it/s]2024-07-25 14:31:48.888832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 14:31:48.888898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factor

In [12]:
import numpy as np
np.load("probs_v0.npy") 

array([[0.08838718, 0.80410892, 0.10750396],
       [0.79719859, 0.07054841, 0.13225305],
       [0.10234115, 0.70382839, 0.19383045],
       [0.17838791, 0.2710644 , 0.55054766],
       [0.06668481, 0.06853338, 0.8647818 ],
       [0.29336137, 0.41247043, 0.2941682 ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.25617051, 0.38137859, 0.36245096],
       [0.26612595, 0.31987479, 0.41399926],
       [0.27281049, 0.16268398, 0.56450552],
       [0.15120441, 0.63815039, 0.21064526],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.83956522, 0.03884743, 0.12158739],
       [0.40776941, 0.2163005 , 0.37593004],
       [0.51514214, 0.10677379, 0.37808412],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.

In [13]:
logits_tta = np.load("probs_v0.npy") + np.load("probs_v1.npy")

In [14]:
logits_combined = logits / 2
logits_combined[:,0] += logits_tta[:,1] / 2
logits_combined[:,1] += logits_tta[:,0] / 2
logits_combined[:,2] += logits_tta[:,2] / 2

In [15]:
test = test.fillna(1/3.)

In [16]:
test

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,fold,full_prompt
0,30192,gpt-4-1106-preview,gpt-4-0613,[Is it morally right to try to have a certain ...,[The question of whether it is morally right t...,"[As an AI, I don't have personal beliefs or op...",1,0,0,0,## Turn 1\n\n### Prompt\nIs it morally right t...
7,370945,gemini-pro,claude-2.0,"[""Bacteria is life on Mars but a heartbeat isn...",[Dune],[This quote seems to be referencing the debate...,0,1,0,0,"## Turn 1\n\n### Prompt\n""Bacteria is life on ..."
13,738614,mpt-30b-chat,gpt4all-13b-snoozy,"[What's the best first move in tic-tac-toe?, T...",[The best first move in tic-tac-toe is often c...,[The best first move in tic-tac-toe is to plac...,1,0,0,0,## Turn 1\n\n### Prompt\nWhat's the best first...
14,862324,vicuna-13b,koala-13b,[Write me a poem in urdu in the style of Iqbal...,"[Jism ko kisi ka paisa nahi,\nWo kisi ka jaan ...",[In the realm of selflessness dwells the true ...,0,0,1,0,## Turn 1\n\n### Prompt\nWrite me a poem in ur...
29,1827787,deepseek-llm-67b-chat,gpt-4-1106-preview,[Given a passage and some supplementary inform...,[Refined passage: \nThe person in the photo is...,[Refined passage: \nThe person in the photo is...,0,0,1,0,## Turn 1\n\n### Prompt\nGiven a passage and s...
...,...,...,...,...,...,...,...,...,...,...,...
502,37132737,gpt-3.5-turbo-0613,pplx-70b-online,[What is the basque country?],[The Basque Country is a region located in the...,[The Basque Country is a region located in the...,0,1,0,0,## Turn 1\n\n### Prompt\nWhat is the basque co...
504,37206390,gpt-4-0613,gpt-4-0125-preview,[what is the song in the movie snatch that pla...,[The song that plays during the hare and dog s...,[The song that plays during the hare coursing ...,1,0,0,0,## Turn 1\n\n### Prompt\nwhat is the song in t...
516,37951752,deepseek-llm-67b-chat,mixtral-8x7b-instruct-v0.1,"[Would you like to play my DnD game?, Ok, make...","[As an AI language model, I'm not capable of p...",[Thank you for the invitation! I'd be happy to...,0,1,0,0,## Turn 1\n\n### Prompt\nWould you like to pla...
519,38558732,llama-2-13b-chat,mistral-7b-instruct,[Could you please explain what does it mean fo...,"[In physics, the concept of time being fundame...","[When time is considered fundamental, it means...",1,0,0,0,## Turn 1\n\n### Prompt\nCould you please expl...


In [17]:
logits_combined
#      ([[0.82734951, 0.07696167, 0.0956888 ],
#        [0.09184527, 0.73428428, 0.17387044],
#        [0.75341365, 0.09070889, 0.1558775 ],
#        [0.28543372, 0.17469092, 0.53987533],
#        [0.06915807, 0.06648109, 0.86436084],

array([[0.82643268, 0.07703017, 0.09653715],
       [0.08829809, 0.74066409, 0.17103785],
       [0.73456872, 0.09576989, 0.16966139],
       [0.27896465, 0.17629373, 0.54474163],
       [0.06694654, 0.06660179, 0.86645165],
       [0.43727775, 0.28652456, 0.27619769],
       [0.52738227, 0.27267491, 0.1999428 ],
       [0.37115289, 0.34354806, 0.28529906],
       [0.33474277, 0.30421737, 0.36103988],
       [0.29249652, 0.28690934, 0.42059416],
       [0.16808184, 0.23067547, 0.60124266],
       [0.63099402, 0.1503617 , 0.21864434],
       [0.32603386, 0.27336882, 0.40059733],
       [0.18497277, 0.2375885 , 0.57743871],
       [0.4029188 , 0.27784361, 0.31923756],
       [0.39152078, 0.20563479, 0.4028444 ],
       [0.03411086, 0.83783454, 0.1280546 ],
       [0.29886857, 0.35408919, 0.34704222],
       [0.12419415, 0.44966961, 0.42613629],
       [0.26620451, 0.33619013, 0.39760533],
       [0.25242854, 0.16163663, 0.58593488],
       [0.54435205, 0.15620757, 0.29944037],
       [0.

In [18]:
test["winner_model_a"] = logits_combined[:,0]
test["winner_model_b"] = logits_combined[:,1]
test["winner_tie"] = logits_combined[:,2]
test = test.fillna(1/3.)
test[["id", "winner_model_a", "winner_model_b", "winner_tie"]].to_csv("submission.csv", index=False)

In [19]:
test

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,fold,full_prompt
0,30192,gpt-4-1106-preview,gpt-4-0613,[Is it morally right to try to have a certain ...,[The question of whether it is morally right t...,"[As an AI, I don't have personal beliefs or op...",0.826433,0.077030,0.096537,0,## Turn 1\n\n### Prompt\nIs it morally right t...
7,370945,gemini-pro,claude-2.0,"[""Bacteria is life on Mars but a heartbeat isn...",[Dune],[This quote seems to be referencing the debate...,0.088298,0.740664,0.171038,0,"## Turn 1\n\n### Prompt\n""Bacteria is life on ..."
13,738614,mpt-30b-chat,gpt4all-13b-snoozy,"[What's the best first move in tic-tac-toe?, T...",[The best first move in tic-tac-toe is often c...,[The best first move in tic-tac-toe is to plac...,0.734569,0.095770,0.169661,0,## Turn 1\n\n### Prompt\nWhat's the best first...
14,862324,vicuna-13b,koala-13b,[Write me a poem in urdu in the style of Iqbal...,"[Jism ko kisi ka paisa nahi,\nWo kisi ka jaan ...",[In the realm of selflessness dwells the true ...,0.278965,0.176294,0.544742,0,## Turn 1\n\n### Prompt\nWrite me a poem in ur...
29,1827787,deepseek-llm-67b-chat,gpt-4-1106-preview,[Given a passage and some supplementary inform...,[Refined passage: \nThe person in the photo is...,[Refined passage: \nThe person in the photo is...,0.066947,0.066602,0.866452,0,## Turn 1\n\n### Prompt\nGiven a passage and s...
...,...,...,...,...,...,...,...,...,...,...,...
502,37132737,gpt-3.5-turbo-0613,pplx-70b-online,[What is the basque country?],[The Basque Country is a region located in the...,[The Basque Country is a region located in the...,0.166018,0.667158,0.166824,0,## Turn 1\n\n### Prompt\nWhat is the basque co...
504,37206390,gpt-4-0613,gpt-4-0125-preview,[what is the song in the movie snatch that pla...,[The song that plays during the hare and dog s...,[The song that plays during the hare coursing ...,0.517965,0.158057,0.323978,0,## Turn 1\n\n### Prompt\nwhat is the song in t...
516,37951752,deepseek-llm-67b-chat,mixtral-8x7b-instruct-v0.1,"[Would you like to play my DnD game?, Ok, make...","[As an AI language model, I'm not capable of p...",[Thank you for the invitation! I'd be happy to...,0.030598,0.899074,0.070327,0,## Turn 1\n\n### Prompt\nWould you like to pla...
519,38558732,llama-2-13b-chat,mistral-7b-instruct,[Could you please explain what does it mean fo...,"[In physics, the concept of time being fundame...","[When time is considered fundamental, it means...",0.557208,0.208581,0.234211,0,## Turn 1\n\n### Prompt\nCould you please expl...
